In [1]:
import inspect
from abc import ABCMeta, abstractmethod
from typing import Dict, Set, Any, List, Optional, Callable, Type

In [2]:
class ContractViolationError(TypeError):
    """Custom exception for API contract violations."""
    pass

In [3]:
class APIRegistry:
    """Global registry for tracking classes that implement specific APIs."""
    
    _registry: Dict[str, Type] = {}
    
    @classmethod
    def register(cls, class_name: str, class_obj: Type) -> None:
        """Register a class in the registry."""
        if class_name in cls._registry:
            raise ContractViolationError(
                f"Class '{class_name}' is already registered. "
                f"Existing: {cls._registry[class_name]}, New: {class_obj}"
            )
        cls._registry[class_name] = class_obj
    
    @classmethod
    def get_registry(cls) -> Dict[str, Type]:
        """Get a copy of the current registry."""
        return cls._registry.copy()
    
    @classmethod
    def get_class(cls, class_name: str) -> Optional[Type]:
        """Retrieve a class from the registry by name."""
        return cls._registry.get(class_name)

In [4]:
class APIContractMeta(ABCMeta):
    """
    Metaclass that enforces API contracts, handles registration,
    and validates class structure.
    """
    
    def __init__(cls, name: str, bases: tuple, namespace: Dict[str, Any]) -> None:
        super().__init__(name, bases, namespace)
        
        # Skip validation for abstract base classes
        if namespace.get('__abstract__', False) or getattr(cls, '__abstract__', False):
            return
        
        # Register the class
        APIRegistry.register(name, cls)
        
        # Enforce API contracts
        cls._enforce_api_contracts()
        
        # Validate class structure
        cls._validate_class_structure()
    
    def _enforce_api_contracts(cls) -> None:
        """Enforce API contracts defined in base classes."""
        for base in cls.__mro__[1:]:  # Skip the class itself
            if hasattr(base, '_api_contract'):
                cls._validate_against_contract(base._api_contract)
    
    def _validate_against_contract(cls, contract: Dict[str, Any]) -> None:
        """Validate that the class satisfies a specific API contract."""
        required_attributes = contract.get('required_attributes', {})
        required_methods = contract.get('required_methods', {})
        
        # Validate required attributes
        for attr_name, attr_type in required_attributes.items():
            if not hasattr(cls, attr_name):
                raise ContractViolationError(
                    f"Class '{cls.__name__}' violates API contract: "
                    f"Missing required attribute '{attr_name}'"
                )
            
            actual_value = getattr(cls, attr_name)
            if not isinstance(actual_value, attr_type):
                raise ContractViolationError(
                    f"Class '{cls.__name__}' violates API contract: "
                    f"Attribute '{attr_name}' must be of type {attr_type.__name__}, "
                    f"got {type(actual_value).__name__}"
                )
        
        # Validate required methods
        for method_name, method_signature in required_methods.items():
            if not hasattr(cls, method_name):
                raise ContractViolationError(
                    f"Class '{cls.__name__}' violates API contract: "
                    f"Missing required method '{method_name}'"
                )
            
            method = getattr(cls, method_name)
            if not callable(method):
                raise ContractViolationError(
                    f"Class '{cls.__name__}' violates API contract: "
                    f"'{method_name}' must be callable"
                )
            
            # Check if it's an abstract method that needs implementation
            if getattr(method, '__isabstractmethod__', False):
                raise ContractViolationError(
                    f"Class '{cls.__name__}' violates API contract: "
                    f"Abstract method '{method_name}' must be implemented"
                )
    
    def _validate_class_structure(cls) -> None:
        """Validate class attributes and methods at runtime."""
        # Check for required class-level attributes
        if hasattr(cls, '_required_attributes'):
            for attr_name in cls._required_attributes:
                if not hasattr(cls, attr_name):
                    raise ContractViolationError(
                        f"Class '{cls.__name__}' is missing required attribute: '{attr_name}'"
                    )
        
        # Validate method signatures if specified
        if hasattr(cls, '_method_signatures'):
            for method_name, expected_params in cls._method_signatures.items():
                if hasattr(cls, method_name):
                    method = getattr(cls, method_name)
                    if callable(method):
                        cls._validate_method_signature(method, method_name, expected_params)
    
    def _validate_method_signature(cls, method: Callable, method_name: str, 
                                 expected_params: List[str]) -> None:
        """Validate that a method has the expected parameters."""
        sig = inspect.signature(method)
        actual_params = list(sig.parameters.keys())
        
        # Remove 'self' from actual parameters for instance methods
        if actual_params and actual_params[0] == 'self':
            actual_params = actual_params[1:]
        
        if actual_params != expected_params:
            raise ContractViolationError(
                f"Method '{method_name}' in class '{cls.__name__}' has incorrect signature. "
                f"Expected parameters: {expected_params}, Got: {actual_params}"
            )

In [5]:
def create_api_contract(required_attributes: Dict[str, type] = None,
                       required_methods: Dict[str, list] = None) -> Dict[str, Any]:
    """
    Helper function to create API contracts for base classes.
    
    Args:
        required_attributes: Dictionary of attribute names and their expected types
        required_methods: Dictionary of method names and their expected parameter lists
    
    Returns:
        API contract dictionary
    """
    return {
        'required_attributes': required_attributes or {},
        'required_methods': required_methods or {}
    }

In [6]:
class DataProcessorContract(metaclass=APIContractMeta):
    """Base class defining the API contract for data processors."""
    
    _api_contract = create_api_contract(
        required_attributes={
            'supported_formats': list,
            'max_file_size': int
        },
        required_methods={
            'process': ['data'],
            'validate': ['data']
        }
    )
    
    # Mark as abstract to skip validation
    __abstract__ = True
    
    @abstractmethod
    def process(self, data: Any) -> Any:
        """Process the data. Must be implemented by subclasses."""
        pass
    
    @abstractmethod
    def validate(self, data: Any) -> bool:
        """Validate the data. Must be implemented by subclasses."""
        pass

In [7]:
class ImageProcessor(DataProcessorContract):
    """Concrete implementation for image processing."""
    
    supported_formats = ['jpg', 'png', 'gif']
    max_file_size = 1024 * 1024  # 1MB
    
    def process(self, data: bytes) -> bytes:
        """Process image data."""
        print(f"Processing image data of {len(data)} bytes")
        return data
    
    def validate(self, data: bytes) -> bool:
        """Validate image data."""
        return len(data) <= self.max_file_size and len(data) > 0

In [8]:
class TextProcessor(DataProcessorContract):
    """Concrete implementation for text processing."""
    
    supported_formats = ['txt', 'csv', 'json']
    max_file_size = 10 * 1024 * 1024  # 10MB
    
    def process(self, data: str) -> str:
        """Process text data."""
        print(f"Processing text: {data[:50]}...")
        return data.upper()
    
    def validate(self, data: str) -> bool:
        """Validate text data."""
        return isinstance(data, str) and len(data) > 0

In [9]:
class AdvancedTextProcessor(TextProcessor):
    """Advanced text processor with additional features."""
    
    encoding = 'utf-8'
    
    def process(self, data: str) -> str:
        """Process text data with encoding handling."""
        try:
            encoded_data = data.encode(self.encoding)
            return super().process(data)
        except UnicodeEncodeError:
            raise ValueError(f"Data cannot be encoded as {self.encoding}")
    
    def validate(self, data: str) -> bool:
        """Validate text data with encoding check."""
        if not super().validate(data):
            return False
        try:
            data.encode(self.encoding)
            return True
        except UnicodeEncodeError:
            return False

In [10]:
class DatabaseConnector(metaclass=APIContractMeta):
    """Another example with different contract requirements."""
    
    _required_attributes = ['host', 'port', 'timeout']
    _method_signatures = {
        'connect': ['username', 'password'],
        'execute_query': ['query', 'parameters']
    }
    
    host = 'localhost'
    port = 5432
    timeout = 30
    
    def connect(self, username: str, password: str) -> bool:
        """Connect to the database."""
        print(f"Connecting to {self.host}:{self.port} as {username}")
        return True
    
    def execute_query(self, query: str, parameters: dict) -> list:
        """Execute a database query."""
        print(f"Executing query: {query} with params: {parameters}")
        return []

In [11]:
def demonstrate_system():
    """Demonstrate the meta-programming system in action."""
    
    print("=== API Contract Meta-Programming System ===\n")
    
    # Show registry
    print("1. Registry Contents:")
    registry = APIRegistry.get_registry()
    for class_name, class_obj in registry.items():
        print(f"   - {class_name}: {class_obj}")
    
    print("\n2. Testing Valid Classes:")
    
    # Test ImageProcessor
    img_processor = ImageProcessor()
    print(f"   ImageProcessor validation: {img_processor.validate(b'test data')}")
    
    # Test TextProcessor
    text_processor = TextProcessor()
    result = text_processor.process("Hello, World!")
    print(f"   TextProcessor result: {result}")
    print(f"   TextProcessor validation: {text_processor.validate('test')}")
    
    # Test AdvancedTextProcessor
    advanced_processor = AdvancedTextProcessor()
    result = advanced_processor.process("Advanced text processing")
    print(f"   AdvancedTextProcessor result: {result}")
    
    # Test DatabaseConnector
    db_connector = DatabaseConnector()
    db_connector.connect("admin", "secret")
    db_connector.execute_query("SELECT * FROM users", {"limit": 10})
    
    print("\n3. Testing Contract Violations:")
    
    try:
        class InvalidProcessor(DataProcessorContract):
            """This class will violate the contract."""
            
            supported_formats = ['pdf']  # Missing max_file_size and required methods
            
    except ContractViolationError as e:
        print(f"   Contract violation caught: {e}")
    
    try:
        class InvalidDBConnector(metaclass=APIContractMeta):
            """This class violates its own required attributes."""
            
            _required_attributes = ['host', 'port']  # But doesn't define them
            
    except ContractViolationError as e:
        print(f"   Contract violation caught: {e}")
    
    try:
        class InvalidMethodSignature(DatabaseConnector):
            """This class violates method signature requirements."""
            
            def connect(self, username: str) -> bool:  # Missing password parameter
                return True
                
    except ContractViolationError as e:
        print(f"   Contract violation caught: {e}")

In [12]:
if __name__ == "__main__":
    demonstrate_system()

=== API Contract Meta-Programming System ===

1. Registry Contents:
   - DatabaseConnector: <class '__main__.DatabaseConnector'>

2. Testing Valid Classes:
   ImageProcessor validation: True
Processing text: Hello, World!...
   TextProcessor result: HELLO, WORLD!
   TextProcessor validation: True
Processing text: Advanced text processing...
   AdvancedTextProcessor result: ADVANCED TEXT PROCESSING
Connecting to localhost:5432 as admin
Executing query: SELECT * FROM users with params: {'limit': 10}

3. Testing Contract Violations:
   Contract violation caught: Class 'InvalidDBConnector' is missing required attribute: 'host'
   Contract violation caught: Method 'connect' in class 'InvalidMethodSignature' has incorrect signature. Expected parameters: ['username', 'password'], Got: ['username']
